# API initialization

The authentification keys are stored as environment variables.

To save authentification keys, do the following in the command prompt (for MacOS):

$touch ~/.bash_profile

Then :

$open -a TextEdit.app ~/.bash_profile

add the following in the opened file (for each variable):

export VARIABLE_NAME=VARIABLE_VALUE

In [6]:
import pprint
import twitter
import os

CONSUMER_KEY = os.environ["CONSUMER_KEY"]
CONSUMER_SECRET = os.environ["CONSUMER_SECRET"]
OAUTH_TOKEN = os.environ["OAUTH_TOKEN"]
OAUTH_TOKEN_SECRET = os.environ["OAUTH_TOKEN_SECRET"]
auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

# Trending tweets

In [6]:
# The Yahoo! Where On Earth ID for the entire world is 1.
# See https://dev.twitter.com/docs/api/1.1/get/trends/place and
# http://developer.yahoo.com/geo/geoplanet/

WORLD_WOE_ID = 1
FR_WOE_ID = 23424819

# To get the rate limits for a given type of request :
# https://dev.twitter.com/rest/public/rate-limits

# Trend request
world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
FR_trends = twitter_api.trends.place(_id=FR_WOE_ID)
FR_trends

[{'as_of': '2016-12-06T18:33:51Z',
  'created_at': '2016-12-06T18:26:58Z',
  'locations': [{'name': 'France', 'woeid': 23424819}],
  'trends': [{'name': '#NommeUnTwittoMinistre',
    'promoted_content': None,
    'query': '%23NommeUnTwittoMinistre',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23NommeUnTwittoMinistre'},
   {'name': '#OmardansTpmp',
    'promoted_content': None,
    'query': '%23OmardansTpmp',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23OmardansTpmp'},
   {'name': '#GrammyNominations',
    'promoted_content': None,
    'query': '%23GrammyNominations',
    'tweet_volume': 325066,
    'url': 'http://twitter.com/search?q=%23GrammyNominations'},
   {'name': '#BattlePhoto',
    'promoted_content': None,
    'query': '%23BattlePhoto',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23BattlePhoto'},
   {'name': '#75ansAFD',
    'promoted_content': None,
    'query': '%2375ansAFD',
    'tweet_volume': None,
    '

# Fetch tweets

In [11]:
import json
q = 'Angela Merkel'
count = 1

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets 
search_results = twitter_api.search.tweets(q=q, count=count) 
statuses = search_results['statuses']
statuses

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Tue Dec 06 19:40:51 +0000 2016',
  'entities': {'hashtags': [{'indices': [67, 75], 'text': 'Germany'}],
   'media': [{'display_url': 'pic.twitter.com/lVspaT9Sq8',
     'expanded_url': 'https://twitter.com/WomenintheWorld/status/806216561895870464/photo/1',
     'id': 806216509475278848,
     'id_str': '806216509475278848',
     'indices': [101, 124],
     'media_url': 'http://pbs.twimg.com/media/CzBBpNNUUAACCXj.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/CzBBpNNUUAACCXj.jpg',
     'sizes': {'large': {'h': 886, 'resize': 'fit', 'w': 1280},
      'medium': {'h': 831, 'resize': 'fit', 'w': 1200},
      'small': {'h': 471, 'resize': 'fit', 'w': 680},
      'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
     'source_status_id': 806216561895870464,
     'source_status_id_str': '806216561895870464',
     'source_user_id': 120153772,
     'source_user_id_str': '120153772',
     'type': 'photo',
     'url': 'https

In [ ]:
# Iterate through 5 more batches of results by following the cursor
for _ in range(5):
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError: # No more results when next_results doesn't exist 
        break
        
    # Create a dictionary from next_results, which has the following form: # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']
    
# Show one sample search result by slicing the list...
print(json.dumps(statuses[0], indent=1))